# Hands-on 7: QAOA


In this notebook we look at how the QLM can be used to efficiently simulate QAOA.

- Step 1: Parametrized circuit
- Step 2: Observables
- Step 3: CombinatorialProblem class
- Step 4: MaxCut with QAOA


## Step 1: Parametrized circuit

Parametrized circuits are an important piece to solve variational algorithms on the QLM.
The goal of this first step is simply to show you how to create and instance them manually.
So you have understanding of the tools used by the functions available on the QLM to solve variational algorithms.

Moreover you could definitely create your own parametrized circuit and use optimization plugins on it.

To do this Step 1 we will use the example of the Hands-on 2:

Create a parametrized circuit with:
+ a single qubit
+ a single gate

In [ ]:
from qat.lang.AQASM import *
prog = Program()
qubits_reg = prog.XXX(XXX)

#Define your variables
theta = prog.XXX(float, "\\theta")

#Apply a gate with a rotation along the Y-axis with a parameter
prog.apply(RY(XXX), XXX[0])

#Create and display the circuit
circuit = XXX.to_circ()
%qatdisplay circuit

Once the parametrized goal is to create a superposition of:
+ 30% to get the state 0
+ 70% to get the state 1

Reminder: An angle of 1.9823 will do the job.

In [ ]:
new_circuit = circuit.XXX({"\\theta": XXX})
%qatdisplay new_circuit

You can now execute your new_circuit to verify the result.

In [ ]:
#Create a Job from the circuit generated by binding the variable
job = new_circuit.to_job()

#Import and create the linear algebra simulator
from qat.qpus import LinAlg
linalgqpu = LinAlg()

#Submit the job to the simulator LinAlg and get the results
result = linalgqpu.submit(job)

#Print the results
for sample in result:
    print("State %s probability %s" % (sample.state, sample.probability))

Let's continue with:
+ two qubits
+ two gates


In [ ]:
from qat.lang.AQASM import *
prog = Program()
qubits_reg = prog.XXX(XXX)

#Define your variables
theta = prog.XXX(float, XXX)
gamma = prog.XXX(float, XXX)

#Apply the two gates with two variables
prog.apply(RY(XXX), qubits_reg[XXX])
prog.apply(RY(XXX), qubits_reg[XXX])

#Create and display the circuit
circuit = prog.to_circ()
%qatdisplay circuit

the result we want is:
+ 24% for |00>
+ 56% for |01>
+ 6%  for |10>
+ 14% for |11>

Reminder: 0.927295 and 1.9823 angles will do it.

In [ ]:
new_circuit = circuit.bind_variables({"\\theta": XXX, "\\gamma": XXX})
%qatdisplay new_circuit

You can now execute your new_circuit to verify the result.

In [ ]:
#Create a Job from the circuit generated by binding the variable
job = new_circuit.to_job()

#Import and create the linear algebra simulator
from qat.qpus import LinAlg
linalgqpu = LinAlg()

#Submit the job to the simulator LinAlg and get the results
result = linalgqpu.submit(job)

#Print the results
for sample in result:
    print("State %s probability %s" % (sample.state, sample.probability))

## Step 2: Observables

Observables are the second important tool used by the QLM to solve efficiently the variational algorithms.

We will take as example a simple observable that counts the number of ones in a quantum state over 5 qubits.

This observable can be written as:

$$ O = \Sigma_i (1 - \sigma_z^i)/2 $$

An observable is initialized with the number of qubits it acts on:

In [ ]:
from qat.core import Observable, Term
nbqbits = 5
one_count = Observable(nbqbits)

New Pauli terms can be added to the observable.

First, we need to write our observable $O$ as a sum of weighted Pauli operators:

$$ O = N/2 - \Sigma_i \frac{1}{2}\sigma_z^i $$



In [ ]:
# The sigma Z terms:
for i in range(nbqbits):
    one_count.add_term(Term(-0.5, "Z", [i]))
# And the constant term:
one_count.constant_coeff += nbqbits/2

We can print our observable to check if it is correct

In [ ]:
print(one_count)

Lets build a simple circuit and approximate the expectation of our observable over its final state.

<table>
    <tr><td>
        <img src="simple.PNG" width="100%"></td>
    </tr>
</table>


In [ ]:
from qat.lang.AQASM import Program, X, CNOT, RX

prog_2_ones = Program()
qbits = prog_2_ones.XXX(nbqbits)
prog_2_ones.apply(X, qbits[XXX])
prog_2_ones.apply(XXX, qbits[XXX], qbits[XXX])
circ_2_ones = prog_2_ones.to_circ()

%qatdisplay circ_2_ones

Once the circuit created, we can create a job with our observable "one_count" and submit it:

In [ ]:
from qat.qpus import LinAlg
qpu = LinAlg()
job = XXX.to_job("OBS", observable=XXX, nbshots=30)
print("Number of ones:", qpu.submit(job).value)

We can do the same thing with a less obvious circuit:

In [ ]:
prog = Program()
qbits = prog.qalloc(5)
for i, qb in enumerate(qbits):
    prog.apply(RX(0.324 * i), qb)
circ = prog.to_circ()

%qatdisplay circ

Once the circuit created, we can create a job with our observable "one_count" and submit it:

In [ ]:
job = circ.to_job("OBS", observable=XXX, nbshots=30)
print("Number of ones:", qpu.submit(job).value)

To decrease the deviation we can increase the number of shots to a thousand for example:

In [ ]:
job = circ.to_job("OBS", observable=XXX, nbshots=XXX)
print("Number of ones:", qpu.submit(job).value)

Or, we can do the simulation with an "infinite" number of shoots since we are doing simulation:

In [ ]:
job = circ.to_job("OBS", observable=XXX)
print("Exact number of ones:", qpu.submit(job).value)

## Step 3: CombinatorialProblem class

The interface of the library is concentrated into a single class `CombinatorialProblem`.

This class allows to:
* declare boolean variables
* add new clauses (i.e boolean formulae) to the final cost function

The goal of this third step is to show you the features of the CombinatorialProblem class.

You first need to import the class CombinatorialProblem from qat.opt when you want to use it:

In [ ]:
from XXX import XXX

With the import of CombinatorialProblem done you can create your problem object using the CombinatorialProblem function:

In [ ]:
# Declaring a fresh problem
my_problem = XXX()

From your problem object you can now create variables:
+ by giving them names
+ and using the function new_var

In [ ]:
# Declaring a new variable
v0 = my_problem.XXX()
v1 = my_problem.XXX()

Or create multiple variables at the same time using new_vars:

In [ ]:
# Or several variables
v_array = my_problem.XXX(4)

Using the following cell you can see that the variables (contained by v0, v1 and v_array) are indexed starting from 0:

In [ ]:
# Variable are indexed starting from 0
print(v0, v1)
print(", ".join(str(v) for v in v_array))

Clauses can be built using variables and boolean operators:
+ | for the OR
+ & for the AND
+ ^ for the XOR
+ ~ for the NOT

In [ ]:
# Clauses are built using boolean operators (|, &, ^, ~) and variables
print(v0 XXX v1)
print(v_array[0] XXX v_array[2])
print(v0 XXX v_array[0])
print(XXXv0)

#And you can combine them
print(~(v0 ^v_array[3] | v1))

To add clauses to your problem you can use the function add_clause from your problem object and specifying the clause you want to add:

In [ ]:
# Clauses are added to a problem using the `add_clause` method
my_problem.XXX(v0 ^ v1)

You can add a weight to your clauses:

In [ ]:
# Clauses can be weighted
my_problem.XXX(v0 | v1, weight=2.)

And clauses can be printed:

In [ ]:
for clause, weight in my_problem.clauses:
    print(clause, weight)

By default, the class assumes that the described problem is a minimization problem.
It is possible to specify maximization problems by adding an argument in the constructor.

In practice, this will simply flip the sign of the cost function (or more precisely, its Hamiltonian encoding).

In [ ]:
my_maximization_problem = CombinatorialProblem(maximization=True)

Once a problem is declared, you can now use qaoa_ansatz to get a job which contain the circuit:

In [ ]:
my_problem = CombinatorialProblem()
variables = my_problem.new_vars(5)
for i in range(4):
    my_problem.add_clause(variables[i]^variables[i+1])

# We just need to specify a number of layers
depth = 3 #Do not make it a too big number to avoid an error in the display part (nevetheless feel free to test)
ansatz = my_problem.XXX(depth).circuit
%qatdisplay ansatz

The variational ansatz is parametrized by abstract variables $\gamma_0,...,\gamma_{l-1}$ and $\beta_0,...,\beta_{l-1}$.

Variables can be listed using get_variables as follows:

In [ ]:
print("Variables:", ansatz.XXX())

You can see that their name is latex compliant, just for a nice display.

It is possible to bind these variables using their names and giving a value like for example np.pi:

In [ ]:
import numpy as np
ansatz_gamma_0_pi = ansatz.bind_variables({"\\gamma_{0}": XXX})
# or equivalently
ansatz_gamma_0_pi = ansatz(**{"\\gamma_{0}": XXX})
%qatdisplay ansatz_gamma_0_pi

In order to be able to generate a QAOA Anstaz the Problem class first encodes each clause into a small Hamiltonian using the following inductive definition:
If boolean clauses are represented using the following grammar:

$exp := exp \lor exp | exp \land exp | exp \oplus exp | \neg exp | V$

Then the Hamiltonian encoding proceeds as follow:

$H(e_1\lor e_2) = H(e_1) + H(e_2) - H(e_1)H(e_2)$

$H(e_1 \land e_2) = H(e_1) * H(e_2)$

$H(e1 \oplus e2) = H(e1) + H(e2) - 2H(e1)H(e2)$

$H(\neg e) = 1 - H(e)$

$H(V(i)) = \frac{1 - \sigma_i^z}{2}$

The complete encoding is then obtained by summing these smaller Hamiltonian (with some eventual coefficients to account for the weights). 

Finally, if the problem is a maximization problem, the sign of the Hamiltonian is flipped, so that the problem becomes a minimization problem.

The Hamiltonian can be obtained using the `.get_observable()` method:

In [ ]:
my_problem = CombinatorialProblem()
variables = my_problem.new_vars(5)
for i in range(4):
    my_problem.add_clause(variables[i]^variables[i+1])
print("Minimization:\n", my_problem.XXX())

my_problem = CombinatorialProblem(maximization=XXX)
variables = my_problem.new_vars(5)
for i in range(4):
    my_problem.add_clause(variables[i]^variables[i+1])
print("Maximization:\n",my_problem.XXX())

Once the observable is generated, there are two distinct circuit synthesis algorithm that can be used to extract an Ansatz from the cost Hamiltonian:

* The "default" algorithm naively produces a subcircuit per term in the Hamiltonian for each layer of the Ansatz. For most applications, this algorithm is enough and will provide a relatively efficient Ansatz.

* The "coloring" heuristics does pretty much the same but optimizes the ordering of the terms in order to minimize circuit depth.

* The "gray_synth" heuristics uses Amy et al phase polynomial synthesis algorithm to implement the entangling portion of the Ansatz. This can help reduce the CNOT count of the resulting circuit.

In [ ]:
my_problem = CombinatorialProblem()
n = 7
variables = my_problem.new_vars(n)
for i in range(n - 2):
    my_problem.add_clause(variables[i] ^ variables[i+1] ^ variables[i+2])
print("Cost Hamiltonian:\n", my_problem.get_observable())
circuit1 = my_problem.qaoa_ansatz(1, strategy="XXX").circuit
circuit2 = my_problem.qaoa_ansatz(1, strategy="XXX").circuit
circuit3 = my_problem.qaoa_ansatz(1, strategy="XXX").circuit

In [ ]:
%qatdisplay circuit1

In [ ]:
%qatdisplay circuit2

In [ ]:
%qatdisplay circuit3

With the CombinatorialProblem class you can build your problem efficiently and the next step will show you on an example how to solve it on the QLM.

## Step 4: MaxCut with QAOA

The `qat.vsolve.qaoa` namespace also contains a very simple wrapper to produce problems describing a MAXCUT instance.

The class can be instantiated using a networkx graph:

In [ ]:
#This cell will create the graph (Nothing to complete here)
#Import networkx 
import networkx as nx

#Create our simple graph
G = nx.Graph()
G.add_nodes_from([0, 1, 2, 3, 4])
G.add_edge(0, 1)
G.add_edge(0, 4)
G.add_edge(1, 2)
G.add_edge(1, 4)
G.add_edge(2, 3)
G.add_edge(3, 4)

#Plot our graph
import matplotlib.pyplot as plt
plt.figure(figsize=(8, 8))
nodes_positions = nx.spring_layout(G, iterations=len(G.nodes())*100)
nx.draw_networkx(G, 
                 pos=nodes_positions, 
                 node_color='#4EEA6A', 
                 node_size=440, 
                 font_size=14)
plt.show()

From the graph created in the previous cell we will create the MaxCut problem.

In [ ]:
from XXX import XXX
problem = MaxCut(XXX)
print(problem)

 To minimize our problem we are going to :
 + use ScipyMinimizePlugin from qat.plugins.
 + create a stack with ScipyMinimizePlugin and our qpu
 + create our job by using qaoa_ansatz with 3 layers
 + submit our job and print the result

In [ ]:
from qat.qpus import get_default_qpu
from XXX import XXX
qpu = get_default_qpu()
stack = ScipyMinimizePlugin(method="COBYLA",
                            tol=1e-2, 
                            options={"maxiter":150}) | XXX
# We can directly call the to_job method of the Problem class to pack an Ansatz and 
# the cost observable in a single abstract Job
job = problem.XXX(1) # Here 1 is the number of layers of the Ansatz
result = stack.submit(XXX)
print("Final energy:", result.value)

If we wish we can print the evolution of the energy during the optimization process:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(eval(result.meta_data["optimization_trace"]))
plt.xlabel("steps")
plt.ylabel("energy")
plt.show()

We can now do the final run with the best parameters obtained during the optimization

In [ ]:
import numpy as np
#Retrieving the optimized parameters:
params = eval(result.meta_data['parameters'])

#Binding the variables:
sol_job = job(**{key: var for key, var in zip(job.get_variables(), params)})

#Checking that this indeeds gives the optimized energy
sol_res = qpu.submit(sol_job)
print("Check, energy =", sol_res.value)

#Rerunning in 'SAMPLE' mode to get the most probable states:
sampling_job = sol_job.circuit.to_job()
sol_res = qpu.submit(sampling_job)
print("Most probable states are:")
for sample in sol_res:
    if sample.probability > 0.05:
        print(sample.state, sample.probability)
# We can also directly cast states into bitstrings for practical use:
print("And as bitstrings:")
for sample in sol_res:
    if sample.probability > 0.05:
        print(sample.state.bitstring,  sample.probability)
        indices_bin_1 = np.where(np.array(list(sample.state.bitstring), dtype=int) == 1)[0]
        indices_bin_0 = np.where(np.array(list(sample.state.bitstring), dtype=int) == 0)[0]
        print("0 list : "+ str(indices_bin_0))
        print("1 list : " + str(indices_bin_1) + "\n")
        
        plt.figure(figsize=(8, 8))
        node_size = 440
        font_size = 14
        nx.draw_networkx(G, 
                         pos=nodes_positions, 
                         nodelist=indices_bin_1.tolist(), 
                         node_color='#FFE033', 
                         node_size=node_size, 
                         font_size=font_size)

        nx.draw_networkx(G, 
                         pos=nodes_positions, 
                         nodelist=indices_bin_0.tolist(), 
                         node_color='#7B9BF2', 
                         node_size=node_size, 
                         font_size=font_size)

        nx.draw_networkx_edges(G, pos=nodes_positions)
        plt.show()


You can if you want change the previous optimizer, namely COYLA

Options:

        - 'Nelder-Mead' :ref:`(see here) <optimize.minimize-neldermead>`
        - 'Powell'      :ref:`(see here) <optimize.minimize-powell>`
        - 'CG'          :ref:`(see here) <optimize.minimize-cg>`
        - 'BFGS'        :ref:`(see here) <optimize.minimize-bfgs>`
        - 'Newton-CG'   :ref:`(see here) <optimize.minimize-newtoncg>`
        - 'L-BFGS-B'    :ref:`(see here) <optimize.minimize-lbfgsb>`
        - 'TNC'         :ref:`(see here) <optimize.minimize-tnc>`
        - 'COBYLA'      :ref:`(see here) <optimize.minimize-cobyla>`
        - 'SLSQP'       :ref:`(see here) <optimize.minimize-slsqp>`
        - 'trust-constr':ref:`(see here) <optimize.minimize-trustconstr>`
        - 'dogleg'      :ref:`(see here) <optimize.minimize-dogleg>`
        - 'trust-ncg'   :ref:`(see here) <optimize.minimize-trustncg>`
        - 'trust-exact' :ref:`(see here) <optimize.minimize-trustexact>`
        - 'trust-krylov' :ref:`(see here) <optimize.minimize-trustkrylov>`